In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
tf.keras.backend.clear_session()

In [8]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [9]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [10]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),...,Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t),Minas Gerais - Consumo de Cimento (t)
0,2008,0.738122,0.717622,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.765604,5.483335e+08,...,0.846543,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497,12196.722756,5902.354000
1,2009,0.737078,0.721167,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.767083,5.302764e+08,...,0.848819,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866,13086.093708,6032.354000
2,2010,0.736297,0.724635,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.768464,5.046425e+08,...,0.850522,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811,13948.809933,6751.429000
3,2011,0.735777,0.728021,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.769750,4.854622e+08,...,0.851665,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000,14478.034725,7134.988000
4,2012,0.735118,0.730837,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770583,4.867661e+08,...,0.852207,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,14505.212690,7144.026000
5,2013,0.735878,0.734823,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.772298,5.225848e+08,...,0.852224,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054,14035.241237,7225.971000
6,2014,0.736117,0.738392,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.773490,5.676413e+08,...,0.851826,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659,13306.331980,6631.735714
7,2015,0.735877,0.740091,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.773781,5.179392e+08,...,0.850979,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753,12588.656354,5836.891371
8,2016,0.730090,0.749468,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778559,4.524144e+08,...,0.862294,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880,12210.552300,5091.098914
9,2017,0.730553,0.750419,3.480263e+08,1.759670e+07,14.339149,2.842415e+08,9.789036,0.778792,4.213689e+08,...,0.861503,20.552463,889.087988,1271.437296,18.605289,13.974131,76.845115,1.808195,12699.687745,4898.010000


In [11]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
0,0.738122,0.717622,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.765604,5.483335e+08,2.546644e+09,...,11.760983,0.846543,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497,12196.722756
1,0.737078,0.721167,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.767083,5.302764e+08,2.679945e+09,...,9.472585,0.848819,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866,13086.093708
2,0.736297,0.724635,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.768464,5.046425e+08,2.891930e+09,...,9.332485,0.850522,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811,13948.809933
3,0.735777,0.728021,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.769750,4.854622e+08,3.091988e+09,...,10.985689,0.851665,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000,14478.034725
4,0.735118,0.730837,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770583,4.867661e+08,3.189506e+09,...,8.159014,0.852207,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,14505.212690
5,0.735878,0.734823,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.772298,5.225848e+08,3.093872e+09,...,7.913561,0.852224,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054,14035.241237
6,0.736117,0.738392,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.773490,5.676413e+08,2.779340e+09,...,10.350442,0.851826,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659,13306.331980
7,0.735877,0.740091,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.773781,5.179392e+08,2.561731e+09,...,12.454398,0.850979,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753,12588.656354
8,0.730090,0.749468,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778559,4.524144e+08,2.516859e+09,...,13.111908,0.862294,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880,12210.552300
9,0.730553,0.750419,3.480263e+08,1.759670e+07,14.339149,2.842415e+08,9.789036,0.778792,4.213689e+08,2.497710e+09,...,9.479812,0.861503,20.552463,889.087988,1271.437296,18.605289,13.974131,76.845115,1.808195,12699.687745


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7225.971000
5     6631.735714
6     5836.891371
7     5091.098914
8     4898.010000
9     5509.999000
10    6256.450000
11    6954.530000
12    6975.675000
13    6706.893000
14            NaN
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
0,1.323232,-1.620866,-1.838149,0.673524,-0.408054,-2.145004,-1.321648,-1.614153,1.204752,-0.990234,...,0.984445,-1.468389,0.546435,-1.229262,-1.471435,-0.707157,0.340929,0.384164,0.760640,-1.478335
1,0.960748,-1.314156,-1.689631,1.223073,-0.189336,-2.066959,-1.288472,-1.285800,0.820657,-0.546688,...,-0.051710,-1.017356,0.681014,-1.970482,-1.167751,-0.184239,0.652313,0.732526,0.855996,-0.395185
2,0.689922,-1.014059,-1.069034,1.286693,1.395717,-0.274561,-1.329227,-0.979439,0.275392,0.158672,...,-0.115145,-0.679745,0.748566,-1.479584,-0.829204,-1.733683,0.662956,1.008690,1.063440,0.655504
3,0.509488,-0.721005,-0.540878,1.182986,1.874830,0.623361,-1.340601,-0.693916,-0.132596,0.824346,...,0.633403,-0.453278,-0.081023,-0.245950,-0.944407,-0.009940,0.941327,1.189931,1.063951,1.300038
4,0.280737,-0.477317,-0.095331,0.897610,1.388579,0.961592,-0.117537,-0.509072,-0.104860,1.148829,...,-0.646476,-0.345909,-1.282156,0.057146,-0.449809,0.790783,0.731347,0.794405,0.743448,1.333138
5,0.544382,-0.132344,0.271919,0.386180,0.533962,0.959671,0.399829,-0.128412,0.657046,0.830615,...,-0.757614,-0.342490,-2.222830,0.603696,0.686946,-0.484821,0.064056,0.058673,0.338865,0.760767
6,0.627589,0.176473,0.539068,-0.237305,-0.405597,0.515649,0.911621,0.136019,1.615453,-0.215960,...,0.345772,-0.421353,-0.696635,0.979423,1.228615,1.884804,-0.001086,-0.360323,0.052022,-0.126960
7,0.544238,0.323529,0.732633,-0.871091,-1.189838,0.081036,1.299648,0.200563,0.558230,-0.940033,...,1.298413,-0.589358,0.119948,1.063393,1.342416,1.710518,0.013713,-0.355502,-0.096007,-1.001006
8,-1.463830,1.134917,0.899544,-1.196495,-1.175976,0.155328,1.209478,1.261018,-0.835563,-1.089341,...,1.596124,1.653000,0.150363,0.918566,1.263379,-0.892435,-0.182621,-0.273294,-0.219681,-1.461493
9,-1.303270,1.217266,0.989637,-1.290727,-0.710864,0.464412,0.692295,1.312653,-1.495939,-1.153056,...,-0.048438,1.496166,1.537521,0.818685,0.932975,-0.448342,-0.021892,-0.122112,-0.847007,-0.865782


In [14]:
reshaped_train_input = dfToInputRNN(train_input)

In [15]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-1.22797939,  1.13807194,  0.80796688, -0.88251021,
         -0.58408809,  0.25850469,  0.40679149,  1.04994977,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  0

In [16]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7225.971000
5     6631.735714
6     5836.891371
7     5091.098914
8     4898.010000
9     5509.999000
10    6256.450000
11    6954.530000
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [17]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    6954.53
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [18]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
12,-1.340422,0.984852,0.49047,-0.775222,-1.103617,-0.390774,0.251054,0.738141,1.108405,2.635823,...,-2.260447,0.453072,-0.236248,-1.371702,-1.653764,-1.48158,-3.104735,-2.983429,-1.314068,1.441741


In [19]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-1.34042219,  0.98485229,  0.49047019, -0.77522176,
         -1.10361718, -0.39077384,  0.25105411,  0.73814139,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  0

In [20]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    6975.675
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [21]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [22]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [23]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[880128428, 2186231799, 4073720728, 1408390435, 3860214454, 2195917222, 1154871140, 372509347, 3032640978, 1952138805, 697335117, 3330873425, 4058459540, 2462811884, 989456518, 2715290255, 2769772060, 3065958008, 3692898721, 270611201, 866538470, 3175474538, 922707810, 2818042881, 3743244320, 958284028, 3459605576, 2410233810, 1212796417, 1272014388, 1867800855, 3134189661, 3853321213, 2565209911, 2761574460, 3025556493, 1629378337, 1049826228, 3778232570, 652506325, 3463832518, 3238735717, 790349055, 3422611311, 2690702617, 4028511370, 3349903814, 697644792, 1705487832, 2061450669, 2804454964, 3879158214, 2660288701, 1772998191, 623939350, 4084109977, 3907681921, 3316233839, 1433428757, 4157528266, 906009142, 3173835721, 3837178888, 608644677, 3210726547, 1597022706, 3496406281, 3362334734, 3027357930, 524611178, 1461789347, 3063440431, 3746704997, 61308826, 1552722754, 2836223330, 3711081617, 324313353, 165159796, 3204547849, 2760145902, 398157283, 2805124048, 1633165462, 2839968460,

2023-09-07 20:01:32.428751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-07 20:01:32.655285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-07 20:01:32.655456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 43220.32421875
winner_seed: 880128428


Step: 1 ___________________________________________
loss: 60957.46484375


Step: 2 ___________________________________________
loss: 35227.77734375
winner_seed: 4073720728


Step: 3 ___________________________________________
loss: 66609.140625


Step: 4 ___________________________________________
loss: 19339.70703125
winner_seed: 3860214454


Step: 5 ___________________________________________
loss: 68305.7265625


Step: 6 ___________________________________________
loss: 41559.23046875


Step: 7 ___________________________________________
loss: 51751.29296875


Step: 8 ___________________________________________
loss: 73080.6796875


Step: 9 ___________________________________________
loss: 37716.1640625


Step: 10 ___________________________________________
loss: 29500.572265625


Step: 11 ___________________________________________
loss: 49019.625


Step: 12 ___________________________________________
loss: 54698.75


Step: 13 _________

In [24]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 40162996.0000
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 38443724.0000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 11648101.0000
Epoch 4/10000
1/1 [==============================] - 0s 5ms/step - loss: 1669070.0000
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 4505039.0000
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 1610126.6250
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 739580.1875
Epoch 8/10000
1/1 [==============================] - 0s 5ms/step - loss: 2763638.7500
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 459891.3438
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 1080431.3750
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 868260.3125
Epoch 12/10000
1/1 [==============================] -

1/1 [==============================] - 0s 4ms/step - loss: 148324.6406
Epoch 97/10000
1/1 [==============================] - 0s 5ms/step - loss: 538564.3750
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 419390.9062
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 190140.4844
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 220199.7344
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 441370.6562
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 261911.3906
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 478218.1562
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 542421.1875
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 502321.8438
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 783621.2500
Epoch 107/10000
1/1 [==============================] - 0s 4ms/

1/1 [==============================] - 0s 4ms/step - loss: 463484.3438
Epoch 191/10000
1/1 [==============================] - 0s 4ms/step - loss: 264879.0000
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 499010.3438
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 492650.2500
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 314401.4062
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 530125.8750
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 429857.9688
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 377248.7188
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 266041.0938
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 384497.5000
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 659459.8125
Epoch 201/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 5ms/step - loss: 562851.0000
Epoch 285/10000
1/1 [==============================] - 0s 4ms/step - loss: 298023.8438
Epoch 286/10000
1/1 [==============================] - 0s 5ms/step - loss: 115393.0312
Epoch 287/10000
1/1 [==============================] - 0s 4ms/step - loss: 234918.0469
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 414086.4062
Epoch 289/10000
1/1 [==============================] - 0s 5ms/step - loss: 177549.7500
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 421804.7500
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 386656.6562
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 254989.2031
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 223545.5781
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 263778.2188
Epoch 295/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 117647.3828
Epoch 379/10000
1/1 [==============================] - 0s 4ms/step - loss: 439069.3438
Epoch 380/10000
1/1 [==============================] - 0s 5ms/step - loss: 179959.1250
Epoch 381/10000
1/1 [==============================] - 0s 4ms/step - loss: 348763.2188
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 306312.7188
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 125479.8359
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 365724.5312
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 400565.1562
Epoch 386/10000
1/1 [==============================] - 0s 5ms/step - loss: 293378.7188
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 232789.1250
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 126095.6484
Epoch 389/10000
1/1 [==============================] - 0s 4

...

1/1 [==============================] - 0s 4ms/step - loss: 601055.6875
Epoch 755/10000
1/1 [==============================] - 0s 4ms/step - loss: 235725.2500
Epoch 756/10000
1/1 [==============================] - 0s 4ms/step - loss: 371205.7812
Epoch 757/10000
1/1 [==============================] - 0s 4ms/step - loss: 156404.2344
Epoch 758/10000
1/1 [==============================] - 0s 4ms/step - loss: 374463.8438
Epoch 759/10000
1/1 [==============================] - 0s 4ms/step - loss: 284300.1562
Epoch 760/10000
1/1 [==============================] - 0s 4ms/step - loss: 245270.3906
Epoch 761/10000
1/1 [==============================] - 0s 4ms/step - loss: 329151.6562
Epoch 762/10000
1/1 [==============================] - 0s 4ms/step - loss: 407265.7500
Epoch 763/10000
1/1 [==============================] - 0s 4ms/step - loss: 252220.1250
Epoch 764/10000
1/1 [==============================] - 0s 4ms/step - loss: 184781.7969
Epoch 765/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 343168.5938
Epoch 849/10000
1/1 [==============================] - 0s 4ms/step - loss: 171904.3281
Epoch 850/10000
1/1 [==============================] - 0s 4ms/step - loss: 271229.9688
Epoch 851/10000
1/1 [==============================] - 0s 4ms/step - loss: 415878.8750
Epoch 852/10000
1/1 [==============================] - 0s 4ms/step - loss: 187796.7500
Epoch 853/10000
1/1 [==============================] - 0s 4ms/step - loss: 650720.6875
Epoch 854/10000
1/1 [==============================] - 0s 4ms/step - loss: 319115.5625
Epoch 855/10000
1/1 [==============================] - 0s 4ms/step - loss: 317569.7812
Epoch 856/10000
1/1 [==============================] - 0s 4ms/step - loss: 259798.9844
Epoch 857/10000
1/1 [==============================] - 0s 4ms/step - loss: 254769.6719
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 369300.4062
Epoch 859/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 238208.2969
Epoch 943/10000
1/1 [==============================] - 0s 4ms/step - loss: 272530.7500
Epoch 944/10000
1/1 [==============================] - 0s 4ms/step - loss: 338057.4688
Epoch 945/10000
1/1 [==============================] - 0s 4ms/step - loss: 96689.6797
Epoch 946/10000
1/1 [==============================] - 0s 4ms/step - loss: 179886.9219
Epoch 947/10000
1/1 [==============================] - 0s 4ms/step - loss: 263549.2812
Epoch 948/10000
1/1 [==============================] - 0s 4ms/step - loss: 197327.7656
Epoch 949/10000
1/1 [==============================] - 0s 4ms/step - loss: 269184.7500
Epoch 950/10000
1/1 [==============================] - 0s 4ms/step - loss: 274386.4688
Epoch 951/10000
1/1 [==============================] - 0s 4ms/step - loss: 221186.4844
Epoch 952/10000
1/1 [==============================] - 0s 4ms/step - loss: 393587.2188
Epoch 953/10000
1/1 [==============================] - 0s 4m

Epoch 1036/10000
1/1 [==============================] - 0s 5ms/step - loss: 347040.6562
Epoch 1037/10000
1/1 [==============================] - 0s 4ms/step - loss: 347042.7500
Epoch 1038/10000
1/1 [==============================] - 0s 4ms/step - loss: 712254.3125
Epoch 1039/10000
1/1 [==============================] - 0s 4ms/step - loss: 282859.4062
Epoch 1040/10000
1/1 [==============================] - 0s 4ms/step - loss: 424776.7812
Epoch 1041/10000
1/1 [==============================] - 0s 4ms/step - loss: 315580.2812
Epoch 1042/10000
1/1 [==============================] - 0s 4ms/step - loss: 234147.6719
Epoch 1043/10000
1/1 [==============================] - 0s 4ms/step - loss: 216980.0000
Epoch 1044/10000
1/1 [==============================] - 0s 4ms/step - loss: 312272.6250
Epoch 1045/10000
1/1 [==============================] - 0s 4ms/step - loss: 323011.5938
Epoch 1046/10000
1/1 [==============================] - 0s 4ms/step - loss: 306940.4688
Epoch 1047/10000
1/1 [==========

1/1 [==============================] - 0s 4ms/step - loss: 130051.8672
Epoch 1130/10000
1/1 [==============================] - 0s 4ms/step - loss: 142383.3750
Epoch 1131/10000
1/1 [==============================] - 0s 4ms/step - loss: 223631.8125
Epoch 1132/10000
1/1 [==============================] - 0s 4ms/step - loss: 340764.0000
Epoch 1133/10000
1/1 [==============================] - 0s 4ms/step - loss: 112852.9688
Epoch 1134/10000
1/1 [==============================] - 0s 4ms/step - loss: 250828.0781
Epoch 1135/10000
1/1 [==============================] - 0s 4ms/step - loss: 470728.2188
Epoch 1136/10000
1/1 [==============================] - 0s 4ms/step - loss: 165180.4688
Epoch 1137/10000
1/1 [==============================] - 0s 4ms/step - loss: 237842.8594
Epoch 1138/10000
1/1 [==============================] - 0s 4ms/step - loss: 107605.9766
Epoch 1139/10000
1/1 [==============================] - 0s 4ms/step - loss: 472157.4062
Epoch 1140/10000
1/1 [===========================

In [25]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 591ms/step
[6975.675](test_target) - [6523.6353](prediction) = 452.0397460937502


In [26]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.06480229455841194

In [27]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [28]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
0,1.279372,-1.229169,-0.918215,-1.407905,-0.838794,-0.751794,-0.482076,-1.238643,1.148205,-1.122043,...,1.412346,-1.279057,-1.335859,1.073554,-1.201994,0.260829,-1.413615,-1.269117,-1.049915,-1.230908
1,-0.117760,0.008898,-0.472376,0.588399,-0.566665,-0.661457,1.392429,0.028285,0.140893,-0.184474,...,-0.643250,0.117025,0.265921,-1.334029,-0.044300,1.073320,0.671180,0.094182,-0.295566,0.012421
2,-1.161611,1.220272,1.390591,0.819506,1.405459,1.413251,-0.910354,1.210357,-1.289098,1.306518,...,-0.769096,1.162032,1.069938,0.260475,1.246294,-1.334149,0.742435,1.174935,1.345481,1.218487


train_target:


,Minas Gerais - Consumo de Cimento (t)
0,6032.354
1,6751.429
2,7134.988


test_input:


...

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Produção de Cimento (t)
12,-1.340422,0.984852,0.49047,-0.775222,-1.103617,-0.390774,0.251054,0.738141,1.108405,2.635823,...,-2.260447,0.453072,-0.236248,-1.371702,-1.653764,-1.48158,-3.104735,-2.983429,-1.314068,1.441741


test_target:


,Minas Gerais - Consumo de Cimento (t)
12,6975.675


1/1 [==============================] - 1s 576ms/step
Error: 452.0397460937502




[7341.32421875,
 7057.666015625,
 7034.328125,
 6118.2412109375,
 5765.40576171875,
 5442.0087890625,
 4981.21142578125,
 5296.07080078125,
 6122.35546875,
 6523.63525390625]

In [29]:
display(targets)
display(predictions)

[7144.026,
 7225.971,
 6631.735714285716,
 5836.891371428574,
 5091.098914285716,
 4898.01,
 5509.999,
 6256.45,
 6954.53,
 6975.675]

[7341.32421875,
 7057.666015625,
 7034.328125,
 6118.2412109375,
 5765.40576171875,
 5442.0087890625,
 4981.21142578125,
 5296.07080078125,
 6122.35546875,
 6523.63525390625]

In [30]:
mae = mean_absolute_error(predictions, targets)
mae

504.1232140624993

In [31]:
porcentage = mae/np.mean(targets)
porcentage

0.08062825375041249